# Sentiment Analysis on Twitter Data

### Importing dependencies


In [5]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

print("Hello")

Hello


Get all the Stopwords

In [6]:
import nltk
nltk.download('stopwords')

print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/akashmahapatra/nltk_data...


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]   Unzipping corpora/stopwords.zip.


### Data Processing

In [19]:
# Reading the data to analyse
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

twitter_data = pd.read_csv('../data/training.1600000.processed.noemoticon.csv', names = column_names, encoding='ISO-8859-1')
twitter_data.shape

twitter_data.head()



,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [20]:
# Counting the missing values from all the columns combined
twitter_data.isnull().sum()


target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [21]:
# Distribution of positive and negative tweets in the dataset
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [28]:
# Check the text where the target is either 0 or 4

negative_twitter_data = twitter_data[twitter_data['target'] == 0]
positive_twitter_data = twitter_data[twitter_data['target'] == 4]

print('===================== Sample Postive data =====================')
print(positive_twitter_data.iloc[0])
print(positive_twitter_data['text'].iloc[0])

print('===================== Sample negative data =====================')
print(negative_twitter_data.iloc[0])
print(negative_twitter_data['text'].iloc[0])

===================== Sample Postive data =====================
target                                               4
id                                          1467822272
date                      Mon Apr 06 22:22:45 PDT 2009
flag                                          NO_QUERY
user                                             ersle
text      I LOVE @Health4UandPets u guys r the best!! 
Name: 800000, dtype: object
I LOVE @Health4UandPets u guys r the best!! 
===================== Sample negative data =====================
target                                                    0
id                                               1467810369
date                           Mon Apr 06 22:19:45 PDT 2009
flag                                               NO_QUERY
user                                        _TheSpecialOne_
text      @switchfoot http://twitpic.com/2y1zl - Awww, t...
Name: 0, dtype: object
@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David 

In [29]:
# Replace target value 4 to 1
twitter_data.replace({'target' : {4:1}},inplace=True)

In [30]:
# Distribution of positive and negative tweets in the dataset
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

| Target Value | Sentiment |
|---|---|
| 0 | Negative |
| 1 | Positive |


**Data Stemming**

In [31]:
port_stem = PorterStemmer()

def stem_function(text):
    stemmed_content =  re.sub('[^a-zA-Z]', ' ', text).lower().split()

    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]

    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [32]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stem_function)

In [33]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [34]:
twitter_data.shape

(1600000, 7)

In [35]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

print(X)

print(Y)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']
[0 0 0 ... 1 1 1]


### Splitting the Data

In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)

print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


### Vectorizing the data set

In [38]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [39]:
print(X_train)

  (0, 909)	0.6747139360858965
  (0, 136685)	0.37838597673035834
  (0, 93777)	0.17608980937731333
  (0, 369241)	0.3167995180387013
  (0, 408437)	0.32456535537405684
  (0, 41294)	0.2805108041255493
  (0, 275283)	0.2935755785635915
  (1, 94705)	0.32962326684791404
  (1, 426834)	0.253704237986435
  (1, 149716)	0.1622007205477998
  (1, 49391)	0.2891031061770205
  (1, 5913)	0.3421504548771284
  (1, 411297)	0.32551559388395285
  (1, 453367)	0.2588091242809426
  (1, 241819)	0.18549356892899846
  (1, 112416)	0.4693063664117002
  (1, 68049)	0.41524348726851457
  (2, 128614)	0.5079437755026398
  (2, 129328)	0.5272831849300049
  (2, 100667)	0.35269918183405957
  (2, 59930)	0.48759610917093593
  (2, 387050)	0.3190907163694111
  (3, 169281)	0.2551178868509669
  (3, 227757)	0.26738662925357076
  (3, 235893)	0.3750776303516075
  :	:
  (1279996, 119923)	0.36052640428764193
  (1279996, 130161)	0.45915421536094353
  (1279996, 43065)	0.49036599854644053
  (1279996, 183737)	0.5639205440693484
  (1279997, 3

In [40]:
print(X_test)

  (0, 65311)	0.34260504576218376
  (0, 132939)	0.30228403152630734
  (0, 138772)	0.2062095249664361
  (0, 149716)	0.14065713180680872
  (0, 150709)	0.46768052007791666
  (0, 182067)	0.2849007149712928
  (0, 274370)	0.19357138253447556
  (0, 275068)	0.39102463301830676
  (0, 343239)	0.2627348207960864
  (0, 375511)	0.23344974321827572
  (0, 393619)	0.2077752363433773
  (0, 416231)	0.26828492776822604
  (1, 75957)	0.5290764438572299
  (1, 200817)	0.37737125257739834
  (1, 369845)	0.5671518024583105
  (1, 374246)	0.4488308189197831
  (1, 435780)	0.2335782165349308
  (2, 53240)	0.2200036780182
  (2, 79378)	0.21111616199737282
  (2, 118098)	0.43306266803959265
  (2, 151706)	0.13632623672324976
  (2, 160763)	0.23880538694434722
  (2, 227275)	0.22442577668624306
  (2, 249310)	0.3809799056080744
  (2, 272290)	0.4492543276567634
  :	:
  (319997, 76191)	0.29850254756445127
  (319997, 84372)	0.2815973625186218
  (319997, 264447)	0.27561887482564695
  (319997, 359479)	0.3103686531466983
  (319997,

### Training the Logistic Regression Model

In [41]:
model = LogisticRegression(max_iter=1000)

In [42]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

### Accuracy Score on Training data

In [43]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [44]:
# Accuracy Score on the training data
print(training_data_accuracy)

0.80446484375


### Accuracy on Test Data

In [45]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [46]:
print(test_data_accuracy)

0.77685


Model accuracy = 77.6 %

### Saving the trained model

In [49]:
import pickle

In [50]:
filename = '../model/trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

### Testing the model

In [51]:
loaded_model = pickle.load(open('../model/trained_model.sav', 'rb'))

In [58]:
X_new = X_test[200]
print(X_new)
print(Y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
    print('Negative Tweet')

else:
    print('Positive Tweet')

  (0, 28948)	0.1910055485608205
  (0, 81276)	0.36525106383404854
  (0, 135333)	0.18001475239210704
  (0, 141935)	0.47858684748725067
  (0, 160208)	0.3755478134948988
  (0, 217113)	0.309205211375475
  (0, 277254)	0.27884058757099656
  (0, 359056)	0.25584081619699683
  (0, 368551)	0.24650356730602532
  (0, 375511)	0.20389040160299737
  (0, 442641)	0.2940611077450335
1
[1]
Positive Tweet


### Exporting the model to PMML Model for Streambase

In [57]:
from joblib import dump

dump(model, '../model/my_model.pmml')

['../model/my_model.pmml']